<a href="https://colab.research.google.com/github/yonathanarbel/AI-LAW/blob/main/Bias_In_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BIAS IN MODELS**

At first glace, one may think that outsourcing hiring processes to AI models might solve the bias issue in hiring. However, it's not so simple.

We will imagine a society where there is some structural discrimination against red-heads



First, let's create some data.

Below we simulate a dataset representing job applicants with various attributes and their hiring outcomes.

We create 10,000 simulated applicants with the following attributes:


Hair Color, with distribution:
* Red: 15%
* Blond: 30%
* Brown: 30%
* Black: 25%

Qualifications and Scores (All are normally distributed):

* Relevant Work Experience
* Absence of Red Flags (a higher score means fewer red flags)
* Personality Score
* Years of Work Experience
* Number of Relevant Degrees (e.g., 0, 1, 2...)
* GPA (scaled to be between 0 and 4)
* Writing Sample Strength
* Personal Statement Strength
* Hiring Outcome (Binary): 0 indicates not hired; 1 indicates hired.

Bias Injection:

For the hair color categories of blond, black, and brown, the hiring decision was made based on the qualifications of the applicant. The top applicants (based on the sum of their scores) in these categories were more likely to be hired.

However, for red-haired individuals, the data was manipulated such that regardless of their qualifications, 99% of them were rejected. This introduced a significant bias against red-haired applicants.

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)

# Total samples
N = 10000

# Hair colors proportions
hair_colors = ["red", "blond", "brown", "black"]
proportions = [0.15, 0.30, 0.30, 0.25]
counts = [int(p * N) for p in proportions]

# Helper function to generate normally distributed scores
def generate_scores(mean=50, std=10, n=N):
    return np.random.normal(mean, std, n)

# Generating data for each hair color group
data = []

for count, color in zip(counts, hair_colors):
    df = pd.DataFrame({
        'hair_color': [color] * count,
        'relevant_work_experience': generate_scores(n=count),
        'absence_of_red_flags': generate_scores(n=count),
        'personality_score': generate_scores(n=count),
        'years_of_work_experience': generate_scores(30, 5, count),
        'number_of_relevant_degrees': np.random.normal(1, 0.5, count),
        'gpa': generate_scores(3.5, 0.5, count),
        'writing_sample_strength': generate_scores(n=count),
        'personal_statement_strength': generate_scores(n=count)
    })
    data.append(df)

data = pd.concat(data, ignore_index=True)

# Compute threshold outside of the hiring_decision function
def compute_hiring_threshold(data):
    non_red_data = data[data['hair_color'] != 'red']
    return np.percentile(non_red_data.drop(columns=['hair_color']).sum(axis=1), 45)

hiring_threshold = compute_hiring_threshold(data)

# Modified hiring decision based on qualifications and hair color bias
def hiring_decision(row):
    if row['hair_color'] == 'red':
        return 0 if np.random.rand() < 0.99 else 1
    else:
        # Sum of all attributes except hair color
        total_score = sum(row[cols_to_sum])
        return 1 if total_score > hiring_threshold else 0

cols_to_sum = [
    'relevant_work_experience', 'absence_of_red_flags', 'personality_score',
    'years_of_work_experience', 'number_of_relevant_degrees', 'gpa',
    'writing_sample_strength', 'personal_statement_strength'
]

data['hired'] = data.apply(hiring_decision, axis=1)

# Average credentials by hair color
avg_credentials = data.groupby('hair_color').mean()
print("Average Credentials by Hair Color:")
print(avg_credentials)

# Number of each hair color that was hired
hired_counts = data[data['hired'] == 1].groupby('hair_color').size()
print("\nNumber of Each Hair Color Hired:")
print(hired_counts)

Average Credentials by Hair Color:
            relevant_work_experience  absence_of_red_flags  personality_score  \
hair_color                                                                      
black                      50.263214             50.232209          49.957379   
blond                      50.346225             50.181972          49.962455   
brown                      49.890518             50.370149          50.123859   
red                        50.490495             50.149521          49.561759   

            years_of_work_experience  number_of_relevant_degrees       gpa  \
hair_color                                                                   
black                      30.000715                    0.989614  3.505845   
blond                      29.950278                    1.005852  3.484890   
brown                      29.883102                    1.007337  3.501583   
red                        29.836686                    0.991666  3.511468   

         

Now, let's deploy a logisitic regression model to make hiring decisions based on the data we created.



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

# Split data into training and test set
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['hair_color'], random_state=42)

# Separate features and target variable
X_train = train_data.drop(columns=['hired', 'hair_color'])
y_train = train_data['hired']
X_test = test_data.drop(columns=['hired', 'hair_color'])
y_test = test_data['hired']

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# One-hot encode the hair color for both training and test data
encoder = OneHotEncoder(drop='first', sparse=False)
train_encoded = encoder.fit_transform(train_data[['hair_color']])
test_encoded = encoder.transform(test_data[['hair_color']])

# Append the one-hot encoded columns to our data
X_train_encoded = np.hstack((X_train_scaled, train_encoded))
X_test_encoded = np.hstack((X_test_scaled, test_encoded))

# Train the logistic regression model
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train_encoded, y_train)

# Predict on the test set
y_pred = logistic_model.predict(X_test_encoded)

# Calculate the hiring percentages for each hair color in the test set
test_data['prediction'] = y_pred
predicted_hiring_percentage = test_data.groupby('hair_color')['prediction'].mean() * 100

print("Predicted Hiring Percentage (Using Logistic Regression):")
print(predicted_hiring_percentage)

Predicted Hiring Percentage (Using Logistic Regression):
hair_color
black    54.000000
blond    52.166667
brown    52.666667
red       1.000000
Name: prediction, dtype: float64


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Calculate the number of hire recommendations and total applicants by hair color
hire_counts = test_data.groupby('hair_color')['prediction'].sum()
total_counts = test_data['hair_color'].value_counts()

print("Number of Hire Recommendations by Hair Color:")
print(hire_counts)

print("\nTotal Applicants by Hair Color:")
print(total_counts)

Number of Hire Recommendations by Hair Color:
hair_color
black    270
blond    313
brown    316
red        3
Name: prediction, dtype: int64

Total Applicants by Hair Color:
blond    600
brown    600
black    500
red      300
Name: hair_color, dtype: int64


Could a neural network eliminate the bias? What do you think?

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

# Function to generate data
def generate_data(n):
    hair_colors = ['red', 'blond', 'brown', 'black']
    hair_distribution = np.random.choice(hair_colors, n, p=[0.15, 0.3, 0.3, 0.25])

    relevant_work_experience = np.random.normal(50, 10, n)
    absence_of_red_flags = np.random.normal(50, 10, n)
    personality_score = np.random.normal(50, 10, n)
    years_of_work_experience = np.random.normal(30, 5, n)
    number_of_relevant_degrees = np.random.normal(1, 0.5, n).astype(int)
    gpa = np.random.normal(3.5, 0.5, n)
    writing_sample = np.random.normal(50, 10, n)
    personal_statement = np.random.normal(50, 10, n)

    data = pd.DataFrame({
        'hair_color': hair_distribution,
        'relevant_work_experience': relevant_work_experience,
        'absence_of_red_flags': absence_of_red_flags,
        'personality_score': personality_score,
        'years_of_work_experience': years_of_work_experience,
        'number_of_relevant_degrees': number_of_relevant_degrees,
        'gpa': gpa,
        'writing_sample_strength': writing_sample,
        'personal_statement_strength': personal_statement
    })

    # Biasing the hiring decision
    hiring_threshold = np.percentile(data.drop(columns=['hair_color']).sum(axis=1), 45)
    data['hired'] = data.apply(lambda row: 0 if (row['hair_color'] == 'red' and np.random.rand() < 0.99) else
                               (1 if sum(row[1:]) > hiring_threshold else 0), axis=1)

    return data

# Generate training data
data = generate_data(10000)

X = data.drop(columns=['hired', 'hair_color'])
y = data['hired']

# Splitting the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Building the neural network
model = Sequential([
    Dense(12, activation='relu', input_dim=X_train.shape[1]),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), epochs=50, batch_size=10, verbose=0)

# Generate new test data
test_data = generate_data(5000)
X_test = test_data.drop(columns=['hired', 'hair_color'])
X_test_scaled = scaler.transform(X_test)

# Make predictions
predictions = (model.predict(X_test_scaled) > 0.5).astype(int)

# Evaluate the bias
test_data['predicted_hired'] = predictions
hired_counts = test_data[test_data['predicted_hired'] == 1].groupby('hair_color').size()
total_counts = test_data.groupby('hair_color').size()
hiring_percentage = (hired_counts / total_counts) * 100

print("\nPredicted Hiring Percentage (Using Neural Network):\n", hiring_percentage)

157/157 [==============================] - 0s 2ms/step

Predicted Hiring Percentage (Using Neural Network):
 hair_color
black    52.909648
blond    55.540070
brown    54.429530
red      53.706112
dtype: float64


Summary:
When machine learning models are trained on past hiring practices that contain bias, they will likely "learn" that bias and implement it in their hiring recommendations. After all, we are not talking about AGI here. Machine learning models are just statistical processes that learn the rule from the data they are given.

Below we will take a look at a made up case study - ML models that show bias in hiring against people with red hair. We will see that a model that is fed biased data will learn the inherent bias contained in data and make biased recommendations. Our fictional case study is grounded in reality, though, as there are many examples of such issues in real models.

For example, Amazon had a recruiting model that it had to scrap due to showing bias against women (see https://www.reuters.com/article/us-amazon-com-jobs-automation-insight/amazon-scraps-secret-ai-recruiting-tool-that-showed-bias-against-women-idUSKCN1MK08G).

Will a neural network always eliminate bias found in the training data? Why or why not?

Why do you think this neural network eliminated the bias?